## 12.내장 자료형 상속과 다중 상속
### 12.1 내장 자료형의 상속은 까다롭다
2.2 버전 이후부터 내장 자료형을 상속할 수 있었지만, C언어로 작성된 내장 클래스의 코드는 사용자가 오버라이드한 코드를 호출하지 않으므로 상당한 주의가 필요하다. 일반적으로 서브클래스에서 오버라이드한 메서드는 같은 객체의 다른 내장 메서드에 의해 결코 호출되지 않는다. 예를 들어 dict의 서브클래스에서 오버라이드한 \_\_getitem\_\_( ) 메서드는 내장된 get( )과 같은 메서드에 의해 호출되지 않는다.

In [1]:
""" [예제 12-1] __setitem__()을 오버라이드한 메서드를 무시하는 내장된 dict의 __init__()과 __update__() 메서드 """

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [2]:
# dict 클래스의 __init__() 메서드는 __setitem__()이 오버라이드되었다는 사실을 무시하므로 'one'의 값을 중복하지 않고 그대로 저장한다.
dd = DoppelDict(one = 1)
print(dd)

{'one': 1}


In [3]:
# [] 연산자는 오버라이드한 __setitem__을 호출하므로 복제된 [2, 2]가 매핑된다.
dd['two'] = 2
print(dd)

{'one': 1, 'two': [2, 2]}


In [4]:
# dict 클래스의 update() 메서드도 오버라이드된 __setitem__() 메서드를 호출하지 않으므로 'three'의 값은 복제되지 않는다.
dd.update(three=3)
print(dd)

{'one': 1, 'two': [2, 2], 'three': 3}


이 문제는 self.get( )이 self.\_\_getitem\_\_( )을 호출하는 경우처럼 객체 안에서 호출할 때 뿐만 아니라, 내장 메서드가 호출하는 다른 클래스 오버라이드된 메서드에서도 발생한다.

In [5]:
""" AnswerDict의 __getitem__()을 지나치는 dict.update() """
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42
    
ad = AnswerDict(a='foo')
print(ad['a']) # 예상한 대로 ad['a']는 42를 반환한다.

42


In [6]:
d = {}
d.update(ad) # d는 평범한 dict 객체이며 여기에서는 ad 객체를 이용해서 갱신한다.
print(d['a']) # dict.update() 메서드는 오버라이드된 AnswerDict.__getItem__() 메서드를 무시한다.
print(d)

foo
{'a': 'foo'}


따라서 dict이나 list, str 등의 내장 자료형은 사용자가 정의한 오버라이드된 메서드를 무시하므로 이 클래스들은 직접 상속하면 에러가 발생하기 쉽다. 내장 자료형보다는 쉽게 확장할 수 있도록 설계된 UserDict, UserList, UserString 등을 사용하는 collenctions 모듈에서 클래스를 상속하는 것이 좋다.

In [7]:
""" [예제 12-3] UserDict을 사용하여 원하는 대로 작동하는 DoppelDict2와 AnswerDict2 """
from collections import UserDict

class DoppelDict2(UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict2(one = 1)
print(dd)
dd['two'] = 2
print(dd)
dd.update(three=3)
print(dd) # 오버라이드된 AnswerDict.__setItem__() 메서드를 호출하여 모두 복제되었다.

{'one': [1, 1]}
{'one': [1, 1], 'two': [2, 2]}
{'one': [1, 1], 'two': [2, 2], 'three': [3, 3]}


In [8]:
class AnswerDict2(UserDict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict2(a='foo')
print(ad['a'])

d = {}
d.update(ad)  
print(d['a']) # 오버라이드된 AnswerDict.__getItem__() 메서드를 호출한다.
print(d)

42
42
{'a': 42}


### 12.2 다중 상속과 메서드 결정 순서
다중 상속을 지원하는 언어에서는 별개의 상위 클래스가 동일한 이름으로 메서드를 구현할 때 발생하는 이름 충돌 문제를 해결해야 한다. 이를 <b>다이아몬드 문제</b>라고 한다. 그러나, 파이썬은 다중 상속을 설정한 순서(예제 12-4에서는 B, C 순)를 따르므로 호출의 모호함이 해결된다. 이 순서를 매서드 결정 순서(MRO<sup>Method Resolution Order</sup>)라고 한다. 클래스에 있는 \_\_mro\_\_() 속성은 현재 클래스부터 object 클래스까지 슈퍼 클래스들의 MRO를 튜플 형태로 저장한다. 

[예제 12-4]에서 d.pong()을 호출하면, 실제 어느 pong() 메서드가 호출될까? C++에서는 이런 모호함을 해결하기 위해 프로그래머가 메서드 앞에 클래스명을 명시해야 한다. [예제 12-5]에서 보는 것처럼 파이썬에서도 클래스명을 직접 명시할 수 있다. 

<p><img src="Figure12-1.png" style="max-width:50%; height:auto;"></p>

In [15]:
""" [예제 12-4] diamond를 구현한 A, B, C, D 클래스의 정의 """
class A:
    x = 1
    def __init__(self):
        self.x = 'x'
    def ping(self):
        print('ping of A:', self)
        
class B(A):
    def pong(self):
        print('pong of B:', self)
        
class C(A):
    def pong(self):
        print('pong of C:', self)        
        
class D(B, C):         # 상속의 순서가 MRO를 결정짓는다.
    def ping(self):
        super().ping() # A.ping(self)로 A의 메서드 직접 호출도 가능하지만 self를 반드시 명시해야 한다. 바인딩되지 않은 메서드에 접근하는 것이기 때문이다.
        print('post-ping:', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)
        
d = D()
d.pong()  # 단순히 d.pong()으로 호출하면 B 클래스의 메서드가 호출된다. 
C.pong(d) # 객체를 인수로 전달해서 슈퍼클래스의 메서드를 직접 호출할 수 있다.

pong of B: <__main__.D object at 0x7f7f404ab438>
pong of C: <__main__.D object at 0x7f7f404ab438>


In [10]:
""" [예제 12-7] pingpong()이 수행한 다섯 번의 호출 """

d = D()
d.pingpong()

# 결과 해석
# 1~2줄 : Class D의 ping을 출력한 것이다.
#   3줄 : Class A의 ping을 출력한 것이다.
#   4줄 : Class B의 pong을 출력한 것이다.
#   5줄 : Class C의 pong을 출력한 것이다.

ping of A: <__main__.D object at 0x7f7f404ef748>
post-ping: <__main__.D object at 0x7f7f404ef748>
ping of A: <__main__.D object at 0x7f7f404ef748>
pong of B: <__main__.D object at 0x7f7f404ef748>
pong of B: <__main__.D object at 0x7f7f404ef748>
pong of C: <__main__.D object at 0x7f7f404ef748>


In [29]:
""" 개인적인 실험 """

class Parent:
    def printa(self):
        print("parent")
    def printb(self):
        self.printa()

class Child(Parent):
    def printa(self):
        print("child")
        
a = Child() # C++의 Virtual 함수와 같이 부모의 클래스에서 오버라이딩한 함수를 호출하는 경우 
a.printb()  # 자식에서 오버라이딩 한 경우 자식의 함수를 자동 호출한다.

b = Parent() 
b.printb()   

Child.printb(a) # 자식으로 형변환

child
parent
child


In [12]:
""" [예제 12-8] 여러 클래스에서 __mro__속성 조사하기 """
print(bool.__mro__)
def print_mro(cls):
    """ MRO를 좀 더 간결하게 출력하는 print_mro() 함수를 정의한다. """
    print(', '.join(c.__name__ for c in cls.__mro__))

print_mro(bool)

import numbers
print_mro(numbers.Integral) # 여러 수치형 ABC이다.

import io
print_mro(io.BytesIO)       # io 모듈에는 여러 구상 클래스와 ABC(...Base가 붙은)가 들어있다. ※ Implementation Class ↔ Abstract Class
print_mro(io.TextIOWrapper) # Open()으로 파일을 열 때 모드에 따라 이진 파일의 경우 BytesIO, 텍스트 파일의 경우 TextIOWrapper 객체가 반환된다.

(<class 'bool'>, <class 'int'>, <class 'object'>)
bool, int, object
Integral, Rational, Real, Complex, Number, object
BytesIO, _BufferedIOBase, _IOBase, object
TextIOWrapper, _TextIOBase, _IOBase, object


### 12.3 실시계에서의 다중 상속
다중 상속은 이해하기 어렵고 불안정하게 설계하기 쉽다. 아직 종합적인 이론체계를 갖추지는 못했지만, 다음의 조언에 따라 뒤엉킨 클래스 그래프가 만들어지는 것을 예방하자.
<ol>
    <p><li>인터페이스 상속과 구현 상속을 구분한다</li></p>
    다중 상속을 다룰 때는 우선 상속 이유를 명확히 하는 것이 도움이 된다. 상속을 하는 이유는 주로 다음과 같다.
    <ul>
        <li>인터페이스 상속은<a href='https://blog.naver.com/dlehdduf28/221175476494'> is-a</a> 관계를 의미하는 서브타입을 생성한다.</li>
        <li>구현 상속은 재사용을 통해 코드 중복을 피한다.</li>
    </ul>
    실제로 이 두 가지 목적을 모두 가진 경우가 많지만, 의도를 명확히 할 수 있을 때는 명백히 표현하는 것이 좋다. 코드 재사용을 위한 상속은 구현에 관련된 것이며, 구성이나 위임으로 대체할 수 있는 경우도 종종 있다. 한편 인터페이스 상속은 프레임워크에서 중추적인 역할을 수행한다.
    <p><li>ABC를 이용해서 인터페이스를 명확히 한다</li></p>
    최신 파이썬에서는 인터페이스를 정의하기 위해 만들어진 클래스를 ABC로 명시한다. 파이썬 3.4 버전부터는 ABC로 명시하기 위해 abc.ABC나 다른 ABC를 상속하는 것을 의미한다.
    <p><li>코드를 재사용하기 위해 <a href='http://brownbears.tistory.com/149'>믹스인</a><sup>Mixin</sup>을 사용한다</li></p>
    'is-a' 관계를 나타내지 않고 서로 관련없는 여러 서브클래스에서 코드를 재사용하기 위해 설계된 클래스는 명시적으로 믹스인 클래스로 만들어야 한다. 개념적으로 믹스인 클래스는 새로운 자료형을 정의하지 않고, 단지 재사용할 메서드들을 묶어놓을 뿐이다. 믹스인 클래스로 객체를 생성하면 안 되며, 믹스인 클래스를 상속하는 구상 클래스는 다른 클래스도 상속해야 한다. 각각의 믹스인 클래스는 밀접히 연관된 메서드 몇 개를 구현해서 하나의 구체적인 행위를 제공해야 한다.
    <p><li>이름을 통해 믹스인임을 명확히 한다</li></p>
    파이썬에서는 믹스인 클래스를 명시하는 공식적인 방법이 없으므로 클래스명 뒤에 Mixin을 붙일 것을 권장한다.
    <p><li>ABC가 믹스인이 될 수는 있지만, 믹스인이라고 해서 ABC인 것은 아니다</li></p>
    ABC는 구상 메서드를 구현할 수 있으므로 믹스인으로도 사용할 수 있다. 그리고 ABC는 자료형을 정의하지만, 믹스인은 자료형을 정의하지 않는다. 게다가 ABC는 다른 클래스의 유일한 기저 클래스가 될 수 있는 반면, 믹스인 하나만 사용해서 서브클래스를 정의하면 안 된다.
    <p><li>두 개 이상의 구상 클래스에서 상속받지 않는다</li></p>
    구상 클래스는 0개 또는 많아야 하나의 구상 슈퍼클래스를 가져야 한다.
    <p><li>사용자에게 집합 클래스를 제공한다</li></p>
    ABC 또는 믹스인을 조합해서 호출 코드에 유용한 기능을 제공할 수 있을 때는 이들을 적절히 통합하는 클래스를 제공하는 것이 좋다. 이런 클래스를 집합클래스<sup>aggregate class</sup>라고 한다.
    <p><li>클래스 상속보다 객체 구성을 사용하라</li></p>
    구성을 좋아하게 되면 설계의 융통성이 향상된다. 
</ol>